### 문제 3-1 :  콘텐츠분쟁해결 RAG 시스템 - 간단 실습 가이드

In [1]:
from dotenv import load_dotenv
import os
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:3])

UPSTAGE_API_KEY = os.getenv("UPSTAGE_API_KEY")
print(UPSTAGE_API_KEY[:5])

gsk
up_HS


In [1]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_upstage import UpstageEmbeddings, ChatUpstage
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

loader = PyPDFLoader("../../data/콘텐츠분쟁해결_사례.pdf")
documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500, 
    chunk_overlap=300, 
    separators=[
        "\n【사건개요】",
        "\n【쟁점사항】",
        "\n【처리경위】",
        "\n【처리결과】",
        "\n■", "\n\n", "\n", ".", " ", ""
    ]
)

docs = text_splitter.split_documents(documents)

embeddings = UpstageEmbeddings(model="solar-embedding-1-large")

vectorstore = FAISS.from_documents(docs, embeddings)
retriever = vectorstore.as_retriever(
    search_type="similarity", 
    search_kwargs={"k": 5}
)

llm = ChatUpstage(
    model="solar-pro",
    base_url="https://api.upstage.ai/v1",
    temperature=0.2
)

prompt_template = """
당신은 콘텐츠 분야 전문 법률 자문가입니다. 
아래 분쟁조정 사례들을 바탕으로 정확하고 전문적인 법률 조언을 제공해주세요.

관련 분쟁사례:
{context}

상담 내용: {question}

답변 가이드라인:
1. 제시된 사례들을 근거로 답변하세요
2. 관련 법령이나 조항이 있다면 명시하세요
3. 비슷한 사례의 처리경위와 결과를 참고하여 설명하세요
4. 실무적 해결방안을 단계별로 제시하세요
5. 사례에 없는 내용은 "제시된 사례집에서는 확인할 수 없습니다"라고 명시하세요

전문 법률 조언:
"""

prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template
)

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff",
    chain_type_kwargs={"prompt": prompt},
    return_source_documents=True
)

test_questions = [
    "온라인 게임에서 시스템 오류로 아이템이 사라졌는데, 게임회사가 복구를 거부하고 있습니다. 어떻게 해결할 수 있나요?",
    "인터넷 강의를 중도 해지하려고 하는데 과도한 위약금을 요구받고 있습니다. 정당한가요?",
    "무료체험 후 자동으로 유료전환되어 요금이 청구되었습니다. 환불 가능한가요?",
    "미성년자가 부모 동의 없이 게임 아이템을 구매했습니다. 환불받을 수 있는 방법이 있나요?",
    "온라인 교육 서비스가 광고와 다르게 제공되어 계약을 해지하고 싶습니다. 가능한가요?"
]

def classify_dispute_type(query: str) -> str:
    game_keywords = ["게임", "아이템", "계정", "캐릭터", "레벨", "길드", "온라인게임"]
    elearning_keywords = ["강의", "온라인교육", "이러닝", "수강", "환불", "화상교육"]
    web_keywords = ["웹사이트", "무료체험", "자동결제", "구독", "사이트"]

    query_lower = query.lower()

    if any(keyword in query_lower for keyword in game_keywords):
        return "게임"
    elif any(keyword in query_lower for keyword in elearning_keywords):
        return "이러닝"
    elif any(keyword in query_lower for keyword in web_keywords):
        return "웹콘텐츠"
    else:
        return "기타"


for q in test_questions:
    result = qa_chain.invoke({"query": q})
    print("="*50)
    print("질문:", q)
    print("분류:", classify_dispute_type(q))
    print("답변:", result["result"])
    print("참조 문서 수:", len(result["source_documents"]))


c:\Users\USER\AppData\Local\pypoetry\Cache\virtualenvs\mylangchain-app-CGunLKE6-py3.11\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


질문: 온라인 게임에서 시스템 오류로 아이템이 사라졌는데, 게임회사가 복구를 거부하고 있습니다. 어떻게 해결할 수 있나요?
분류: 게임
답변: ### 전문 법률 조언: 온라인 게임 시스템 오류로 인한 아이템 복구 거부 시 해결 방안

#### 1. **사례 분석을 통한 법적 쟁점 정리**
제시된 사례들을 종합하면, 시스템 오류로 인한 아이템 복구 문제는 다음과 같은 쟁점으로 분류됩니다:
- **(1) 시스템 오류 입증 가능성**: 게임사의 기술적 결함 또는 이용자 과실(예: 개인 PC 문제) 여부 (2009년 사례 참조)
- **(2) 계정 소유권 확인**: 명의자와 실소유자가 다른 경우 복구 거부 가능성 (2006년 사례 참조)
- **(3) 게임사의 약관 및 책임 범위**: 프로그램 오류 인정 시 복구 의무 발생 (2006년 프로그램 오류 사례 참조)
- **(4) 아이템의 법적 성격**: 게임머니/아이템이 "금전" 또는 "디지털 재산"으로 볼 수 있는지 (민법 제250조, 251조 참조)

---

#### 2. **관련 법령 및 약관 검토**
- **민법 제250조(도품·유실물 특례)**:  
  - 아이템이 "금전"이 아닌 경우, 피해자(유실자)는 2년 내 반환 청구 가능.  
  - 단, 게임사가 "공개시장 역할"을 한 경우(민법 제251조), 선의의 구매자(신청인)에게 대가 변상 후 회수 가능 (2007년 해킹 아이템 사례 참조).  
  - **단, 게임머니는 일반적으로 "금전"으로 해석될 수 있어 복구 권리 제한 가능성 있음**.

- **게임산업진흥에 관한 법률 제12조(사업자의 준수사항)**:  
  - 게임사는 시스템 오류로 인한 이용자 피해에 대해 **신의성실 원칙**에 따라 조치해야 함.  
  - 단, 약관에서 명시적으로 책임을 제한하는 경우(예: "시스템 오류로 인한 손실은 복구하지 않음") 법적 효력 발생 가능.

- **약관의 규제에 관한 법률 제6조(불공정 약관 무효)**:  
  - 게임사의 약관이 지나치게 이용자에게 불리한